In [2]:
import autocompleter 
autocompl = autocompleter.Autocompleter()

In [3]:
df = autocompl.import_json("name.json")
df.shape, df.columns

load json file...
(22264, 3)


((22264, 3), Index([&#39;IsFromCustomer&#39;, &#39;Text&#39;, &#39;index&#39;], dtype=&#39;object&#39;))

In [4]:
df.head()

,IsFromCustomer,Text,index
0,True,Hi! I placed an order on your website and I ca...,0
1,True,I think I used my email address to log in.,0
2,True,My battery exploded!,1
3,True,"It's on fire, it's melting the carpet!",1
4,True,What should I do!,1


# Data Selection and Cleaning

The data is going to separate the threads from the customer and the representative, separate the sentenses based on the punctuation (we will keep the punctuation), the final text will be cleaned up with some light regex and only the sentense larger than 1 word will be kept.

Finally, since the representative has the tendency to ask the same question over and over again, the autocomplete is extremely useful by suggesting a complete sentense. In our case, we will count the number of occurence of the same sentense so we can use it as a feature later on and delete the duplicates.

In [5]:
new_df = autocompl.process_data(df)
new_df.shape, new_df.columns

select representative threads...
split sentenses on punctuation...
Text Cleaning using simple regex...
calculate nb words of sentenses...
count occurence of sentenses...
remove duplicates (keep last)...
(8560, 5)


((8560, 5),
 Index([&#39;IsFromCustomer&#39;, &#39;Text&#39;, &#39;index&#39;, &#39;nb_words&#39;, &#39;Counts&#39;], dtype=&#39;object&#39;))

# Model and TFIDF matrix

A matrice of similarity is calculated based on the frequency of all the words in the data using tfidfvectorizer

In [6]:
model_tf, tfidf_matrice = autocompl.calc_matrice(new_df)

tfidf_matrice  (8560, 99397)


# Ranking Function

Finally, the autocomplete is calculating the similarity between the sentense in the data and the prefix of the sentense written by the representative. As a weight feature, we chose to reorder using the frequency of the most common similar sentense.

examples of auto completions

In [7]:
prefix = 'What is your'

print(prefix,"    \n ")

autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

What is your     
 


[&#39;What is your account number?&#39;,
 &#39;What is your order number?&#39;,
 &#39;What is your phone number?&#39;]

In [8]:
prefix = 'How '
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

How       


[&#39;How does that sound?&#39;, &#39;How may I help you?&#39;, &quot;How&#39;s the 7th?&quot;]

In [9]:
prefix = 'Let me'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

Let me      


[&#39;Let me investigate&#39;, &#39;Let me assist you&#39;, &#39;Let me look&#39;]

In [10]:
prefix = 'how was'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

how was      


[&#39;How does that sound?&#39;, &#39;How may I help you?&#39;, &quot;How&#39;s the 7th?&quot;]

Now, without any uppercase and just with the important words...

In [11]:
prefix = 'hello'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

hello      


[&#39;Hello wernzio 2&#39;,
 &#39;Hello I can assist you with this&#39;,
 &#39;Hello werner how are things going?\\&#39;]